Training, Validation and testing dataset were labeled by batch manually

In [1]:
import sys
sys.path.append('/host/d/Github')
import os
import numpy as np
import pandas as pd
import nibabel as nb
import whole_heart_segmentation_ZC.functions_collection as ff

In [6]:
main_path = '/host/d/Data/WHS/current_challenge/original_data'

# select batches for training, validation
#### here, we need to split both CT and MR into 5 bachces respectively


In [14]:
def make_batches(n_samples, n_batches=5,shuffle = None, seed=None):
    """
    先 shuffle，然后把 index 分成 n_batches 个 batch。
    """

    assert shuffle in [True, False], "shuffle must be True or False"

    if seed is not None:
        np.random.seed(seed)

    # step 1: shuffle
    if shuffle:
        indices = np.random.permutation(n_samples)   # 生成 0~n_samples-1 并打乱顺序
    else:
        indices = np.arange(n_samples)

    # step 2: average split
    batch_list = np.zeros(n_samples, dtype=int)
    batch_size = n_samples // n_batches
    remainder = n_samples % n_batches

    start = 0
    for i in range(n_batches):
        size = batch_size + (1 if i == n_batches - 1 and remainder > 0 else 0)
        batch_list[indices[start:start+size]] = i
        start += size
    
    return batch_list.tolist()


In [16]:
modality_list = ['CT','MR']
random_seed = 40

In [17]:
results = []
basic_info = pd.read_excel('/host/d/Data/WHS/Patient_lists/original_data_info.xlsx', dtype = {'patient_id': str})
for modality_num in range(len(modality_list)):
    modality_name = modality_list[modality_num]
   
    cases = ff.find_all_target_files(['center*/*'],os.path.join(main_path,modality_name))
    num_of_cases = len(cases)
    print('Total number of {} cases: {}'.format(modality_name, num_of_cases))
    batch_list = make_batches(num_of_cases, n_batches=5, shuffle=True, seed=random_seed)

    for n in range(0,num_of_cases):

        batch = batch_list[n]
        case_path = cases[n]

        case_id = os.path.basename(case_path)
        center = os.path.basename(os.path.dirname(case_path))

        row = basic_info[(basic_info['modality']==modality_name) & 
                         (basic_info['center']==center) & 
                         (basic_info['patient_id']==case_id)]

        
        results.append([batch, modality_name, center, case_id,
                        int(row['size_x']), int(row['size_y']), int(row['size_z']),
                        float(row['spacing_x']), float(row['spacing_y']), float(row['spacing_z'])])
            
df = pd.DataFrame(results, columns=['batch','modality','center','patient_id',
                                    'size_x','size_y','size_z',
                                    'spacing_x','spacing_y','spacing_z'])
ff.make_folder([os.path.join('/host/d/Data/WHS/Patient_lists')])
df.to_excel(os.path.join('/host/d/Data/WHS/Patient_lists','train_val_batch_list.xlsx'), index=False)

Total number of CT cases: 40
Total number of MR cases: 46


# Make patient list for train and testing

In [19]:
batch_list = pd.read_excel(os.path.join('/host/d/Data/WHS/Patient_lists','train_val_batch_list.xlsx'), dtype = {'patient_id': str})
main_path = '/host/d/Data/WHS/current_challenge/original_data'

img_list = []
label_list = []
for i in range(len(batch_list)):
    modality = batch_list.loc[i,'modality']
    center = batch_list.loc[i,'center']
    patient_id = batch_list.loc[i,'patient_id']
    
    img_path = os.path.join(main_path, modality, center, patient_id, 'image.nii.gz')
    label_path = os.path.join(main_path, modality, center, patient_id, 'label.nii.gz')
    
    img_list.append(img_path)
    label_list.append(label_path)

batch_list['img_path'] = img_list
batch_list['label_path'] = label_list
batch_list.to_excel(os.path.join('/host/d/Data/WHS/Patient_lists','train_val_path_list.xlsx'), index=False)
    